In [1]:
import pandas as pd

### Cache data - price, prododuct data etc
suppoedly it caches itself in data.py, but that logic broke

In [2]:
from data.data import PriceData, ProductData, BenchmarkData
from config import Benchmarks, START_DATE

In [ ]:
# .9s
benchmark_data = BenchmarkData()
tickers = benchmark_data.get_constituents(Benchmarks.SP500)
len(tickers)

In [ ]:
# 1m20s
product_data = ProductData()
data = product_data.get_data(tickers=tickers)
len(data)

In [ ]:
# 32.4s
price_data = PriceData()
data = price_data.get_data(tickers, start_date=START_DATE, end_date='2025-06-01')
len(data)

### Backtesting

In [ ]:
# run date by date - slower
from backtesting.backtest import Backtest
from backtesting.scenarios import Scenario
from config import DEFAULT_BACKTEST_PARAMS, DEFAULT_STRATEGY_CONFIG

scenario = Scenario(
    name=DEFAULT_BACKTEST_PARAMS["scenario_name"],
    start_date=DEFAULT_BACKTEST_PARAMS["start_date"],
    end_date=DEFAULT_BACKTEST_PARAMS["end_date"],
    constraints=DEFAULT_BACKTEST_PARAMS["constraints"],
    portfolio_setup=DEFAULT_BACKTEST_PARAMS["setup"],
    benchmark=DEFAULT_BACKTEST_PARAMS["benchmark"],
)

strategies = DEFAULT_STRATEGY_CONFIG.to_strategies()
scenario.set_strategies(strategies)

backtest = Backtest(scenario)
backtest.run(verbose=True)


In [ ]:
# run all dates at once - faster
from backtesting.backtest import Backtest
from backtesting.scenarios import Scenario
from config import DEFAULT_BACKTEST_PARAMS, DEFAULT_STRATEGY_CONFIG

scenario = Scenario(
    name=DEFAULT_BACKTEST_PARAMS["scenario_name"],
    start_date=DEFAULT_BACKTEST_PARAMS["start_date"],
    end_date=DEFAULT_BACKTEST_PARAMS["end_date"],
    constraints=DEFAULT_BACKTEST_PARAMS["constraints"],
    portfolio_setup=DEFAULT_BACKTEST_PARAMS["setup"],
    benchmark=DEFAULT_BACKTEST_PARAMS["benchmark"],
)

strategies = DEFAULT_STRATEGY_CONFIG.to_strategies()
scenario.set_strategies(strategies)

backtest = Backtest(scenario)
backtest.run_batch(verbose=True)


### Generate report
only a backtest object generates report, by first generating the analytics...

In [ ]:
# backet -> get analytics
# backtest -> generate report with analytics object
backtest.generate_report(filename="test")
backtest.generate_report(filename="test2")

### Gridsearch

In [4]:
from backtesting.grid_search import GridSearch
from backtesting.scenarios import Scenario
from config import DEFAULT_PORTFOLIO_SETUP, SIMPLE_CONSTRAINTS, Benchmarks

from config import Strategies

grid_search_params = [
        {"triggers": [Strategies.MACD_CROSSOVER], "filter": []},
        {"triggers": [Strategies.RSI_CROSSOVER], "filter": []},
        {
            "triggers": [Strategies.MACD_CROSSOVER],
            "filter": [Strategies.BOLLINGER_BANDS],
        },  # mean reversion confirmed by bollinger bands
        {
            "triggers": [Strategies.RSI_CROSSOVER, Strategies.Z_SCORE_MEAN_REVERSION],
            "filter": [],
        },  # momentum with mean reversion
        {
            "triggers": [
                Strategies.MACD_CROSSOVER,
                Strategies.BOLLINGER_BANDS,
                Strategies.Z_SCORE_MEAN_REVERSION,
                Strategies.RSI_CROSSOVER,
            ],
            "filter": [],
        },  # DON'T KNOW
    ]

scenario = Scenario(
    name="test_gs_1yr",
    start_date="2022-01-01",
    end_date="2022-12-31",    
    constraints=SIMPLE_CONSTRAINTS,
    portfolio_setup=DEFAULT_PORTFOLIO_SETUP,
    benchmark=Benchmarks.SP500,
    portfolio_name="sp500_no_short_1k_monthly",
)
gs = GridSearch(scenario, 10)
gs.set_grid_params(grid_search_params)

In [ ]:
gs.run()

In [ ]:
gs.results_to_dataframe()

In [ ]:
gs.get_grid_search_schedule()